<a href="https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/Toonify_yourself_ttmai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toonify yourself!

Αυτή είναι μία ελαφρώς απλοποιημένη έκδοση του notebook που παρέχεται από το original project:
https://www.justinpinkney.com/toonify-yourself/


Πριν ξεκινήσουμε, βεβαιωθείτε ότι έχετε επιλέξει το **GPU** σαν *Runtime Type*.

(Runtime --> change runtime type --> Hardware accelerator: GPU)

# Αρχικά πατάμε τα παρακάτω κουμπάκια για να κατεβάσουμε τον κώδικα της εφαρμογής και να ετοιμάσουμε το περιβάλλον μας.

In [ ]:
%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/justinpinkney/stylegan2
%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run

In [ ]:
!mkdir raw
!mkdir aligned
!mkdir generated

## Πατήστε το παρακάτω κουμπάκι για να ανεβάσετε την εικόνα σας.

Δεν χρειάζεται να κάνετε κάποια επεξεργασία στην εικόνα σας, αλλά για βέλτιστο αποτέλεσμα, είναι καλό η εικόνα σας να έχει ανάλυση μεγαλύτερη από 1024x1024 pixels.


In [ ]:
from google.colab import files 
uploaded = files.upload() 
for photo in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=photo, length=len(uploaded[photo])))

## Κατεβάζουμε τα εκπαιδευμένα μοντέλα

In [ ]:
import pretrained_networks

blended_url = "https://drive.google.com/uc?id=1L3ek9B4GJ7pv-QcCFK52hqn5DFxRpg6o" 
ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

_, _, Gs_blended = pretrained_networks.load_networks(blended_url)
_, _, Gs = pretrained_networks.load_networks(ffhq_url)

In [ ]:
!mv {photo} raw/.
!python align_images.py raw aligned

In [ ]:
!python project_images.py --num-steps 550 aligned generated

## Μετατροπή εικόνας σε Cartoon

In [ ]:
import numpy as np
from PIL import Image
import dnnlib
import dnnlib.tflib as tflib
from pathlib import Path

latent_dir = Path("generated")
latents = latent_dir.glob("*.npy")
for latent_file in latents:
  latent = np.load(latent_file)
  latent = np.expand_dims(latent,axis=0)
  synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  images = Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs)
  Image.fromarray(images.transpose((0,2,3,1))[0], 'RGB').save(latent_file.parent / (f"{latent_file.stem}-toon.jpg"))


## Προβολή της εικόνας
Σημείωση. Παρακάτω θα δείτε να παρουσιάζονται 2 εικόνες.

Η πρώτη εικόνα είναι ο τρόπος που η εφαρμογή ανασυνθέτει το πρόσωπο της φωτογραφίας που του δώσαμε με δεδομένα που έχει απο το εκπαιδευμένο μοντέλο του.

Μετά, αφού έχει μάθει να συνθέτει το πρόσωπο μας, δίνει τις οδηγίες για να παράγει την καρτουνίστικη εκδοχή που θέλουμε.

In [ ]:
from IPython.display import Image 
embedded = Image(filename="generated/" + photo.split(".")[0] + "_01.png", width=256)
display(embedded)
tooned = Image(filename="generated/" + photo.split(".")[0] + "_01-toon.jpg", width=256)
display(tooned)